In [1]:
import os
import nibabel as nib
import numpy as np

from modules.Dataset import FeTADataSet
from modules.Utils import get_file_names

In [ ]:
""" Remove zero slices in axial orientation. Also permute the axis of mri images. First axis will be used for axial.
"""

files = get_file_names("feta_2.1/")

for sub, (path_image, path_mask) in files.items():
        # Load nifti images.
        images = nib.load(path_image).get_fdata()
        masks = nib.load(path_mask).get_fdata()
        
        # Permute images axis to get axial view in the first dimension.
        images = np.transpose(images, (2, 0, 1))
        masks = np.transpose(masks, (2, 0, 1))
        
        # Find zero slices.
        indexes = np.where(np.all(np.all(np.array(masks)==False, axis=2)==True, axis=1))[0] 
        images = np.delete(images, indexes, axis=0)
        masks = np.delete(masks, indexes, axis=0) 
        
        # Convert images in numpy to nifti.
        images = nib.Nifti1Image(images, np.eye(4))
        masks = nib.Nifti1Image(masks, np.eye(4))
        
        # Only change the main folder name. Rest of the paths will be same.
        new_path_image = os.path.join("data", *path_image.split('/')[1:])
        new_path_mask = os.path.join("data", *path_mask.split('/')[1:])
        
        # Create sub path.
        new_folder_path = os.path.join(*new_path_image.split('/')[:-1])
        if not os.path.exists(new_folder_path):
            os.makedirs(new_folder_path)
        
        # Save new paths.
        nib.save(images, new_path_image)
        nib.save(masks, new_path_mask)
        

In [ ]:
val = FeTADataSet("val", path="data/", transform=None)

In [ ]:
im = train[0][0]
mask = train[0][1]

In [ ]:
def convert_to_tuple(dataset):
    new_dataset = ()
    
    for i in range(len(dataset)):
        data = tuple(zip(dataset[i][0], dataset[i][1]))
        new_dataset += data
        
    return new_dataset

In [ ]:
x = convert_to_tuple(val)